In [ ]:
import os
import glob
import pandas as pd
import plotly.express as px
import numpy as np
import configparser
import pathlib
from tqdm.auto import tqdm

In [ ]:
# Load config file and set parameters
config = configparser.ConfigParser()
config.read('config.ini')
DATA_FOLDER = config["DEFAULT"]["DATA_FOLDER"]
LOG2FOLDCHANGE_COLUMN = config["DEFAULT"]["LOG2FOLDCHANGE_COLUMN"]
PADJ_COLUMN = config["DEFAULT"]["PADJ_COLUMN"]
GENENAME_COLUMN = config["DEFAULT"]["GENENAME_COLUMN"]
LOG2FOLDCHANGE_THRESHOLD = float(config["DEFAULT"]["LOG2FOLDCHANGE_THRESHOLD"])
PVALUE_THRESHOLD = float(config["DEFAULT"]["PVALUE_THRESHOLD"])

In [ ]:
# find all xls or xlsx files in DATA_FOLDER
files = [file for file in glob.glob(os.path.join(DATA_FOLDER, "*.xls*"))]

# Remove duplicates with different (.xlsx and .xls) endings and keep only the .xlsx
files = list({os.path.splitext(os.path.basename(f))[0]: f for f in files}.values())

print(f"Processing {len(files)} files in {DATA_FOLDER}:")
for file_path in files:
    print(f"   • {os.path.basename(file_path)}")
print(f"Using column '{LOG2FOLDCHANGE_COLUMN}' as log2-fold-change column.")
print(f"Using column '{PADJ_COLUMN}' as adjusted-p-value column.")
print(f"Using column '{GENENAME_COLUMN}' as gene name column.")

In [ ]:
def create_volcano_plot(excel_file_name, log2foldchange_column="log2FoldChange", padj_column="padj", genename_column="gene_name", log2foldchange_threshold=1, pvalue_threshold=0.05):
    """
    Creates a volcano plot from differential expression data stored in .xls or .xlsx files.

    This function reads the input Excel file containing gene expression data, computes
    log-transformed p-values, assigns colors to represent upregulated, downregulated,
    and neutral genes based on log2-fold-change and adjusted p-values (padj), and then
    generates an interactive volcano plot. The plot is saved as an HTML file.

    Parameters:
    -----------
    excel_file_name : str
        The name of the Excel file (with path if needed) containing the data.
        The file can be in '.xls' or '.xlsx' format.

    log2foldchange_column : str, optional
        The name of the column in the Excel file representing the log2-fold-change values
        (default is "log2FoldChange").

    padj_column : str, optional
        The name of the column in the Excel file representing the adjusted p-values
        (default is "padj").

    padj_column : str, optional
        The name of the column in the Excel file representing the gene name
        (default is "gene_name").

    log2foldchange_threshold : float, optional
        Threshold for log2 fold-change to classify genes as upregulated or downregulated 
        (default: 1).

    pvalue_threshold : float, optional
        Threshold for adjusted p-values to identify significant genes (default: 0.05).

    Processing:
    -----------
    - The Excel file is read into a Pandas DataFrame.
    - A new 'color' column is created to categorize genes as 'UP', 'DOWN', or 'NO' based on
      the log2-fold-change and adjusted p-value:
          UP:   log2FoldChange > 1 & p-value < 0.05
          DOWN: log2FoldChange < 1 & p-value < 0.05
          NO:   all other
    - The negative log10 of the adjusted p-values (-log10(padj)) is calculated.
    - A scatter plot is created using Plotly, with genes colored according to their regulation
      status (upregulated, downregulated, or not significant).
    - Vertical and horizontal threshold lines are added to the plot, indicating fold-change
      and significance cutoffs.
    - The plot legend includes counts of 'UP', 'DOWN', and 'NO' genes.
    - The resulting volcano plot is saved as an HTML file in the same directory as the input file.

    Returns:
    --------
    None
        The function generates an HTML file for the volcano plot.

    Example:
    --------
    create_volcano_plot("differential_expression_data.xlsx")
    """

    # load dataframe
    suffix = pathlib.Path(excel_file_name).suffix
    if suffix == ".xls":
        # for old Excel formats
        df = pd.read_table(excel_file_name)
        df.to_excel(("".join([excel_file_name.removesuffix(pathlib.Path(excel_file_name).suffix), ".xlsx"])))
    else:
        df = pd.read_excel(excel_file_name, engine="openpyxl")

    if log2foldchange_column not in df.columns:
        raise ValueError(f"Column '{log2foldchange_column}' not found in the Excel file. Please check the config.ini and make sure the column exists in the Excel file.")
    if padj_column not in df.columns:
        raise ValueError(f"Column '{padj_column}' not found in the Excel file. Please check the config.ini and make sure the column exists in the Excel file.")
    if genename_column not in df.columns:
        raise ValueError(f"Column '{genename_column}' not found in the Excel file. Please check the config.ini and make sure the column exists in the Excel file.")

    # add new columns for UP, DOWN, NO and -log10(padj)
    df['color'] = 'NO'
    df.loc[(df[log2foldchange_column] < -log2foldchange_threshold) & (df[padj_column] < pvalue_threshold), 'color'] = 'DOWN'
    df.loc[(df[log2foldchange_column] > log2foldchange_threshold) & (df[padj_column] < pvalue_threshold), 'color'] = 'UP'
    df['-log10(padj)'] = -np.log10(df[padj_column])

    # create volcano-plot
    color_discrete_map = {'UP': 'firebrick', 'DOWN': 'forestgreen', 'NO': 'lightgray'}
    fig = px.scatter(
        df,
        x="log2FoldChange",
        y="-log10(padj)",
        color='color',
        color_discrete_map=color_discrete_map,
        labels={'color': f'padj<{pvalue_threshold:.4g} <br>|log2FoldChange|>{log2foldchange_threshold:.4g}'},
        custom_data=[genename_column],
        template="plotly_white",
        title=os.path.basename(excel_file_name).removesuffix(pathlib.Path(excel_file_name).suffix),
        render_mode="svg",
    ).update_traces(
        hovertemplate="<b>%{customdata[0]}</b><br>"+"<extra></extra>",
        # "-log10(padj): %{y:.2f}<br>" +
        # "log2FoldChange: %{x:.2f}",
    )

    # add counts of UP, DOWN and NO to legend
    newnames = {val: f'{val} {df.color.value_counts()[f"{val}"]}' for val in df.color.unique()}
    fig.for_each_trace(lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name])
    ))

    # add vertical and horizontal lines
    fig.add_vline(x=-log2foldchange_threshold, line_width=3, line_dash="dash", line_color="lightgray")
    fig.add_vline(x=log2foldchange_threshold, line_width=3, line_dash="dash", line_color="lightgray")
    fig.add_hline(y=-np.log10(pvalue_threshold), line_width=3, line_dash="dash", line_color="lightgray")

    fig2 = px.scatter(
        df.loc[(df[log2foldchange_column] < -log2foldchange_threshold) & (df[padj_column] < pvalue_threshold) | (df[log2foldchange_column] > log2foldchange_threshold) & (df[padj_column] < pvalue_threshold)],
        x="log2FoldChange",
        y="-log10(padj)",
        # labels={'color': f'padj<{pvalue_threshold:.4g} <br>|log2FoldChange|>{log2foldchange_threshold:.4g}'},
        render_mode="svg",
        text=genename_column,
    ).update_traces(marker_opacity=0, textposition='top center')

    fig.add_trace(fig2.data[0].update(name='Gene Names', visible='legendonly', showlegend=True))
    fig.for_each_trace(lambda trace: trace.update(marker=dict(size=0, color="white")) if trace.name == 'Gene Names' else ())

    fig.show()
    fig.write_html(".".join([excel_file_name.removesuffix(pathlib.Path(excel_file_name).suffix), "html"]))

In [ ]:
print("Creating volcano plots:")
for file in tqdm(files):
    create_volcano_plot(file, LOG2FOLDCHANGE_COLUMN, PADJ_COLUMN, GENENAME_COLUMN, LOG2FOLDCHANGE_THRESHOLD, PVALUE_THRESHOLD)